In [185]:
import checklist
from checklist.editor import Editor
import numpy as np
editor = Editor()
ret = editor.template('{first_name} is {a:profession} from {country}.',
                       profession=['lawyer', 'doctor', 'accountant'])
np.random.choice(ret.data, 3)

array(['Joe is a doctor from Suriname.',
       'Harold is an accountant from Bosnia and Herzegovina.',
       'Billy is an accountant from Russia.'], dtype='<U67')

In [186]:
ret = editor.template('This is {a:adj} {mask}.',  
                      adj=['good', 'bad', 'great', 'terrible'])
ret.data[:3]

Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at roberta-base and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


['This is a good idea.', 'This is a good example.', 'This is a good sign.']

In [240]:
insurance_type = ["travel", "motor", "life", "term", "health/medicare", "disability", "renters", "home", "housing", "long term care", "property"]
insurance_action = ["renew", "buy", "compare", "find", "claim status", "file claim", "request quote", "quote"]
insurance_names = ["coverage", "insurance", "policy", "insurance policy"]
change_synonyms = ['change', 'provide', 'clarify', 'update', 'confirm', 'check', 'reset']
insurance_information = ["maturity policy", "policy", "returns", "maturity terms", "date"]
policy_terms = [x.lower() for x in ["premium","Actual cash value (ACV)","Agreed value","Stated Value","renewal","Expiry","Maturity","Maturity amount","beneficiary","Deductible","Exclusion","Lapse","Mitigation","Lien"]]
client_credentials = ["phone number", "address", "form details", "contact", "mobile number", "email", "email id"]
coverage_types = ["Comprehensive","Collision","Natural Calamity","Bodily Injury","Medical Payments","Riot","Theft"]

In [237]:
template_info = [
    {"templates":"{mask} {mask} {verb:insurance_action} for {insurance_type} {insurance_names} {mask}", "labels": "ask_for_info"},
    {"templates":"{verb:insurance_action} {mask} to {insurance_type} {insurance_names}", "labels":"ask_for_info"},
    {"templates":"I want to {mask} {p1:policy_terms} of {insurance_type} {insurance_names}","labels": "ASK_DEEPAK"},
    {"templates":"When {mask} {mask} {insurance_names} mature?","labels": "ask_for_info"},
    {"templates":"I {mask} to file {mask} {mask} {insurance_type} {insurance_names} claim","labels": "ask_for_info"},
    {"templates":"What {mask} {mask} my claim status?","labels": "ask_for_info"},
    {"templates":"{mask} {policy_terms} {insurance_names}","labels": "ask_for_info"},
    {"templates":"{change_synonyms} {client_credentials} and {mask}","labels": "give_info"},
    {"templates":"{change_synonyms} {mask} {client_credentials}","labels": "give_info"},
    {"templates":"{mask} to {change_synonyms} my communication {client_credentials}","labels": "give_info"},
]

In [238]:
def get_insurance_generations(template, nsamples=10):
    return editor.template(
        **template,
        insurance_type=insurance_type,
        insurance_action=insurance_action,
        insurance_names=insurance_names,
        policy_terms=policy_terms,
        change_synonyms=change_synonyms,
        client_credentials=client_credentials,
        remove_duplicates=True,
        nsamples=nsamples,
        meta=True,
    )


filled_in_info = get_insurance_generations(template_info[1], nsamples=10)
filled_in_info.data, filled_in_info.meta, filled_in_info.labels

['Smiles',
 'Emirates',
 'Cashback',
 'Skywards',
 'Rewards',
 'visa charges',
 'Etihad',
 'Skyward',
 'Booking.com',
 'Spice']

In [433]:
account_types = [
    "savings",
    "salary",
    "ADIB",l
    "Ghina",
    "current",
    "recurring deposit",
    "fixed deposit",
    "loan",
    "demat",
]
qualifiers = ["I", "we"]
loan_types = ["car", "travel", "personal", "student", "education", "housing", "home"]
debt_synonyms = ["financing", "debt", "loan", "finance"]
card_synonyms = ["card", "covered card", "credit card", "debit card", "atm card"]
card_brands = [
    "Smiles",
    "Emirates",
    "Cashback",
    "Skywards",
    "Rewards",
    "Etihad",
    "Skyward",
    "Booking.com",
    "Spice",
]

account_info_terms = [
    "features",
    "interest rates",
    "charges",
    "details",
    "fees",
    "balance",
]
debt_info_terms = [
    "amount due",
    "due amount",
    "payment date",
    "installment",
]
action = ["apply", "get", "open"]
close = ["nearest", "nearby"]
find_close = ["how close is the", "where is the", "need to go to", "ADIB"]
find_what = [
    "ATM",
    "branch",
    "branches",
    "loan agent",
    "cash deposit machine",
    "cheque deposit machine",
    "cash withdrawal",
    "ccdm",
    "cash withdraw machine",
]
inform = [
    "What is my",
    "show me my",
    "find",
    "can you show me the",
    "show",
    "my",
    "show me the",
    "looking for my",
    "Check",
    "Remaining",
    "I want to know",
    "What is",
    "get",
    "Tell me",
    "Where is your"
    "What are the available",
]

In [500]:
templates_info_banking = [
    {
        "templates": "{find_close} {mask} {close} {find_what}",
        "labels": "ask_for_info",
    },
    {
        "templates": "{inform} {mask} {account_types} {pron:mask} {account_info_terms}",
        "labels": "ask_for_info",
    },
    {
        "templates": "{inform} {account_types} {account_info_terms}",
        "labels": "ask_for_info",
    },
    {"templates": "{inform} {cards} {debt_info_terms}", "labels": "ask_for_info"},
    {"templates": "{loan_types} {debt_synonyms}", "labels": "ask_for_info"},
    {"templates": "{action} {card_brands} {cards}", "labels": "{action}"},
    {
        "templates": "How {verb:mask} {qualifiers} {action} {pron:mask} {card_brands} {cards}",
        "labels": "action",
    },
    {"templates": "{action} {account_types} account", "labels": "action"},
    {"templates": "{mask} {mask} open {account_types} account", "labels": "open"},
    {
        "templates": "{account_types} account {account_info_terms}",
        "labels": "ask_for_info",
    },
]

In [501]:
def get_banking_generations(template, nsamples=10):
    return editor.template(
        **template,
        account_types=account_types,
        qualifiers=qualifiers,
        loan_types=loan_types,
        debt_synonyms=debt_synonyms,
        close=close,
        action=action,
        card_brands=card_brands,
        cards=cards,
        find_close=find_close,
        inform=inform,
        find_what=find_what,
        debt_info_terms=debt_info_terms,
        account_info_terms=account_info_terms,
        remove_duplicates=True,
        nsamples=nsamples,
        meta=True,
    )


filled_in_info = get_banking_generations(templates_info_banking[-1], nsamples=15)
filled_in_info.data, filled_in_info.meta, filled_in_info.labels

(['Where is your cash deposit machine',
  'Where is your cash withdraw machine',
  'Where is your branch',
  'Where is your cash deposit machine',
  'Where is your ATM',
  'Where is your loan agent',
  'Where is your branches',
  'Where is your ATM',
  'Where is your cash deposit machine',
  'Where is your branches',
  'Where is your ccdm',
  'Where is your ccdm',
  'Where is your cash withdraw machine',
  'Where is your cash withdrawal',
  'Where is your loan agent'],
 [{'find_what': 'cash deposit machine'},
  {'find_what': 'cash withdraw machine'},
  {'find_what': 'branch'},
  {'find_what': 'cash deposit machine'},
  {'find_what': 'ATM'},
  {'find_what': 'loan agent'},
  {'find_what': 'branches'},
  {'find_what': 'ATM'},
  {'find_what': 'cash deposit machine'},
  {'find_what': 'branches'},
  {'find_what': 'ccdm'},
  {'find_what': 'ccdm'},
  {'find_what': 'cash withdraw machine'},
  {'find_what': 'cash withdrawal'},
  {'find_what': 'loan agent'}],
 ['ask_for_info',
  'ask_for_info',
 

In [497]:
import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
editor = Editor()

dataset = ['This was a very nice movie directed by John Smith.',
           'Mary Keen was brilliant.',
          'I hated everything about this.',
          'This movie was very bad.',
          'I really liked this movie.',
          'just bad.',
          'amazing.',
          ]
t = Perturb.perturb(dataset, Perturb.add_typos)
test2 = INV(**t)